# IsCyclic

In [1]:
import pickle

import torch
from torch_geometric.loader import DataLoader

from GNN import GCN_IsCyclic
from preprocessing import create_isCyclic_dataset

torch.manual_seed(12345)

## Data

In [2]:
dataset, data_objs = create_isCyclic_dataset(saved=True)
print(dataset[0].x)
print(dataset[0].edge_index)
print(dataset[0].y)

Dataset size:  951
['../../../data/IsCyclic/raw/dataset.pkl', '../../../data/IsCyclic/raw/labels.pkl']


Processing...
Done!


tensor([[1.],
        [1.],
        [1.]])
tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 0, 2, 0, 1]])
tensor(1., dtype=torch.float64)


In [3]:
with open("../../../data/IsCyclic/index.pkl", "rb") as file:
    index = pickle.load(file)

In [4]:
train_dataset = dataset[index['idx_train']]
val_dataset = dataset[index['idx_val']]
test_dataset = dataset[index['idx_test']]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
for data in train_loader:
    print(data)
    break

DataBatch(x=[1910, 1], edge_index=[2, 4970], y=[64], batch=[1910], ptr=[65])


## Model

In [6]:
model = GCN_IsCyclic(
    in_features=dataset.num_node_features,
    h_features=64,
)

In [7]:
print(model)

GCN_IsCyclic(
  (conv1): GraphConvolution (1 -> 64)
  (conv2): GraphConvolution (64 -> 64)
  (conv3): GraphConvolution (64 -> 64)
  (dense1): Linear(in_features=64, out_features=16, bias=True)
  (dense2): Linear(in_features=16, out_features=8, bias=True)
  (dense3): Linear(in_features=8, out_features=2, bias=True)
)


## Train

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()

def train():
    model.train()
    for data in train_loader: # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y.long())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

def test(loader):
    model.eval()
    correct = 0
    for data in loader: # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch).squeeze()
        pred = out.argmax(dim=-1)
        correct += int((pred == data.y.long()).sum())
    return correct / len(loader.dataset)

In [9]:
best_test_acc = 0.0
for epoch in range(1, 101):
    train()
    train_acc = test(train_loader)
    val_acc = test(val_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')
    if val_acc >= best_test_acc:
        best_test_acc = val_acc
        best_model_params = model.state_dict()
        print("Checkpoint saved!")

Epoch: 001, Train Acc: 0.5516, Val Acc: 0.5739
Checkpoint saved!
Epoch: 002, Train Acc: 0.7747, Val Acc: 0.7217
Checkpoint saved!
Epoch: 003, Train Acc: 0.7832, Val Acc: 0.7826
Checkpoint saved!
Epoch: 004, Train Acc: 0.9158, Val Acc: 0.8957
Checkpoint saved!
Epoch: 005, Train Acc: 0.9579, Val Acc: 0.9565
Checkpoint saved!
Epoch: 006, Train Acc: 0.6000, Val Acc: 0.6348
Epoch: 007, Train Acc: 0.9284, Val Acc: 0.9304
Epoch: 008, Train Acc: 0.9432, Val Acc: 0.9304
Epoch: 009, Train Acc: 0.9516, Val Acc: 0.9652
Checkpoint saved!
Epoch: 010, Train Acc: 0.9347, Val Acc: 0.9304
Epoch: 011, Train Acc: 0.9516, Val Acc: 0.9478
Epoch: 012, Train Acc: 0.9937, Val Acc: 0.9913
Checkpoint saved!
Epoch: 013, Train Acc: 0.9832, Val Acc: 0.9826
Epoch: 014, Train Acc: 1.0000, Val Acc: 0.9913
Checkpoint saved!
Epoch: 015, Train Acc: 0.9979, Val Acc: 0.9913
Checkpoint saved!
Epoch: 016, Train Acc: 0.9979, Val Acc: 0.9913
Checkpoint saved!
Epoch: 017, Train Acc: 0.9937, Val Acc: 0.9913
Checkpoint saved!
Epo

## Eval

In [10]:
best_model_params = torch.load("iscyclic_weights.pt")
model.load_state_dict(best_model_params)
model.eval()

GCN_IsCyclic(
  (conv1): GraphConvolution (1 -> 64)
  (conv2): GraphConvolution (64 -> 64)
  (conv3): GraphConvolution (64 -> 64)
  (dense1): Linear(in_features=64, out_features=16, bias=True)
  (dense2): Linear(in_features=16, out_features=8, bias=True)
  (dense3): Linear(in_features=8, out_features=2, bias=True)
)

In [11]:
for dataset in ['train', 'val', 'test']:
    acc = test(eval(f"{dataset}_loader"))
    print(f"{dataset} accuracy: {100 * acc:.2f} %")

train accuracy: 99.37 %
val accuracy: 99.13 %
test accuracy: 99.35 %


## Save weight

In [12]:
torch.save(best_model_params, "iscyclic_weights.pt")

## Rough